In [8]:
import math
import hashlib

class BloomFilter:
    def __init__(self, n, p):
        self.size = self.get_size(n, p)
        self.hash_count = self.get_hash_count(self.size, n)
        self.bit_array = 0  # Using integer to represent bit array

    def get_size(self, n, p):
        """
        Calculate the size of bit array(m) to use using
        the formula: m = -(n * ln(p)) / (ln(2)^2)
        """
        m = -(n * math.log(p)) / (math.log(2) ** 2)
        return int(m)

    def get_hash_count(self, m, n):
        """
        Calculate the number of hash functions(k) to use using
        the formula: k = (m/n) * ln(2)
        """
        k = (m / n) * math.log(2)
        return int(k)

    def _hashes(self, item):
        """
        Generate k hash values for the given item using double hashing.
        """
        hash1 = int(hashlib.sha256(item.encode('utf-8')).hexdigest(), 16)
        hash2 = int(hashlib.md5(item.encode('utf-8')).hexdigest(), 16)
        for i in range(self.hash_count):
            yield (hash1 + i * hash2) % self.size

    def add(self, item):
        for hash_val in self._hashes(item):
            self.bit_array |= 1 << hash_val

    def check(self, item):
        for hash_val in self._hashes(item):
            if not self.bit_array & (1 << hash_val):
                return False
        return True


In [9]:
import random

total_profiles = 100000
false_positive_rate = 0.01

profile_ids = random.sample(range(0, total_profiles * 10), total_profiles)

bloom = BloomFilter(n=total_profiles, p=false_positive_rate)

for pid in profile_ids:
    bloom.add(str(pid))

test_profiles = [random.randint(0, total_profiles * 10) for _ in range(1000)]

true_positives = 0
false_positives = 0
true_negatives = 0
false_negatives = 0

profile_ids_set = set(profile_ids)

for pid in test_profiles:
    is_present = pid in profile_ids_set
    bloom_result = bloom.check(str(pid))

    if is_present and bloom_result:
        true_positives += 1
    elif not is_present and bloom_result:
        false_positives += 1
    elif not is_present and not bloom_result:
        true_negatives += 1
    elif is_present and not bloom_result:
        false_negatives += 1

# Display Results
print(f"\nMembership Test Results:")
print(f"Total Tests: {len(test_profiles)}")
print(f"Actual Present: {true_positives}")
print(f"Actual Absent: {1000 - true_positives}")
print(f"Bloom Positive: {true_positives + false_positives}")
print(f"Bloom Negative: {true_negatives + false_negatives}")
print("-" * 30)
print(f"True Positives: {true_positives}")
print(f"False Positives: {false_positives}")
print(f"True Negatives: {true_negatives}")
print(f"False Negatives: {false_negatives}")  # Should be 0


Membership Test Results:
Total Tests: 1000
Actual Present: 100
Actual Absent: 900
Bloom Positive: 109
Bloom Negative: 891
------------------------------
True Positives: 100
False Positives: 9
True Negatives: 891
False Negatives: 0
